In [1]:
import numpy as np
import pandas as pd
import tqdm

In [2]:
"""
train data 또는 test data 중 feature를 추가하고 싶은 데이터를 불러오면 됩니다.!
(train과 test data를 합친 data를 불러와도 됩니다!)
"""
# TODO : 원하는 데이터의 경로를 설정해주세요
#raw_data = pd.read_csv("/opt/ml/input/data/train_data.csv")
raw_data = pd.read_csv("/opt/ml/input/data/test_data.csv")

user_id = raw_data["userID"].unique() # 유저 아이디 저장

In [3]:
raw_data # 원본 확인

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623
...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244


In [4]:
##-------------------대분류 Feature 추가--------------------------
main_category = list()
for i in tqdm.tqdm(range(len(raw_data))) :
    main_category.append(raw_data["testId"].iloc[i][2])
raw_data["bigClass"] = main_category

100%|██████████| 260114/260114 [00:02<00:00, 106535.32it/s]


In [5]:
raw_data # 대분류 추가 이후

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,bigClass
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626,5
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626,5
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625,5
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625,5
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623,5
...,...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832,4
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832,4
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244,4
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244,4


In [6]:
"""
acc_by_big_category
    - key : 유저 아이디
    - value : 해당 유저의 대분류별 정답률 (인덱스가 대분류를 나타낸다. ex. index=1 -> 대분류 1)
    - default value : 기본적으로 0.5라는 값이 들어간다. 
"""
acc_by_big_category = dict()
for user in user_id :
    acc_by_big_category[user] = [0.5]*10

In [7]:
for user in tqdm.tqdm(user_id) : # 모든 유저에 대해
    data = raw_data[raw_data["userID"]==user] # 각 유저에 대한 행만 추출하여 data에 저장
    bc_l = data["bigClass"].unique() # 행들에 있는 고유한 대분류 추출 ex) [6, 2, 8]

    for i in bc_l : # 유저가 풀어본 각 대분류에 대해 
        big_cate = int(i) 
        big_correct = 0
        big_total = 0
        for j in range(len(data)) : # 전체 데이터를 순회하면서 
            if data["bigClass"].iloc[j] == i : # 동일한 대분류에 대해 
                if data["answerCode"].iloc[j] == 1 : # 정답을 맞췄으면, correct, total count + 1
                    big_correct+=1
                    big_total+=1
                else : 
                    big_total+=1 # 맞추지 못했으면 total count만 + 1
        acc_by_big_category[user][big_cate]=(big_correct/big_total) # 전체 데이터를 순회하면서 알게된 대분류 정답률을 저장


100%|██████████| 744/744 [00:09<00:00, 80.32it/s] 


In [8]:
"""
acc_by_big_category_list != acc_by_big_category
: 해당 리스트는 feature를 추가하기 전에 feature 값들을 일렬로 저장하기 위해 만든 리스트입니다.
"""
acc_by_big_category_list = list() 

for i in tqdm.tqdm(range(len(raw_data))) : # 모든 행(instance)에 대해
    u = raw_data["userID"].iloc[i] # 유저를 확인하고
    bc = raw_data["bigClass"].iloc[i] # 대분류를 확인하고
    acc = acc_by_big_category[u][int(bc)] # 해당 유저의 해당 대분류에 대한 정답률을 위에서 정의한 dictionary에서 찾아서
    acc_by_big_category_list.append(acc) # 리스트에 저장한다.

raw_data["bigClassAcc"]=acc_by_big_category_list # 리스트에 bigClassAcc feature 추가

100%|██████████| 260114/260114 [00:04<00:00, 52594.75it/s]


In [9]:
raw_data # 대분류, 대분류별 정답률 추가 이후

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,bigClass,bigClassAcc
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626,5,0.654292
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626,5,0.654292
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625,5,0.654292
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625,5,0.654292
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623,5,0.654292
...,...,...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832,4,0.687500
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832,4,0.687500
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244,4,0.687500
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244,4,0.687500


In [10]:
"""
acc_cate_list : 각 대분류 정답률을 카테고리로 만들어 저장하려고 한 리스트. 
ex. 원래 값 [0.04, 0.79, 0.43, 0.99] -> 곱하기 10 : [0.4, 7.9, 4.3, 9.9] -> round : [0, 8, 4, 10] -> 더하기 1 : [1, 9, 5, 11]
"""
acc_cate_list = list()
for i in range(len(raw_data)) :
    acc = raw_data["bigClassAcc"].iloc[i] * 10
    acc = round(acc) + 1  
    acc_cate_list.append(acc) 

raw_data["bigClassAccCate"] = acc_cate_list # 리스트로 저장

In [11]:
raw_data # 대분류, 대분류별 정답률, 대분류별 정답률 카테고리 추가 후

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,bigClass,bigClassAcc,bigClassAccCate
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626,5,0.654292,8
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626,5,0.654292,8
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625,5,0.654292,8
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625,5,0.654292,8
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623,5,0.654292,8
...,...,...,...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832,4,0.687500,8
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832,4,0.687500,8
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244,4,0.687500,8
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244,4,0.687500,8


In [99]:
# TODO : 이름과 경로 원하는 데로 변경하기
raw_data.to_csv("test_data_with_bigClassAccCate.csv", index=False) # 최종 결과 csv로 반환